In [1]:
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Create Dash app
app = dash.Dash(__name__)

# Clear the layout and do not display exceptions until callback gets executed
app.config.suppress_callback_exceptions = True

# Read the wildfire data into a pandas dataframe
df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/Historical_Wildfires.csv')

# Extract year and month from the 'Date' column
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()  # Converts to month names
df['Year'] = pd.to_datetime(df['Date']).dt.year  # Extracts year

# Unique years for dropdown options
year_options = [{"label": str(year), "value": year} for year in sorted(df['Year'].unique())]

# Layout Section of Dash
app.layout = html.Div(children=[
    # TASK 2.1: Add the Title to the Dashboard
    html.H1('Australia Wildfire Dashboard', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 26}),

    # Outer division
    html.Div([
        # First inner division for adding dropdown helper text
        html.Div([
            html.H2("Select a Region:", style={'marginRight': '10px'}),
            
            # TASK 2.2: Radio items for selecting the region
            dcc.RadioItems(
                options=[
                    {"label": "New South Wales", "value": "NSW"},
                    {"label": "Victoria", "value": "VIC"},
                    {"label": "Queensland", "value": "QLD"},
                    {"label": "South Australia", "value": "SA"},
                    {"label": "Western Australia", "value": "WA"},
                    {"label": "Tasmania", "value": "TAS"},
                    {"label": "Northern Territory", "value": "NT"}
                ],
                value="NSW",
                id='region-selector',
                inline=True
            ),
        ]),

        # Dropdown to select year
        html.Div([
            html.H2("Select a Year:", style={'marginRight': '10px'}),
            dcc.Dropdown(
                id='year-selector',
                options=year_options,
                value=df['Year'].min(),  # Default to the earliest year in the dataset
                placeholder="Select a Year",
                clearable=False
            ),
        ], style={'marginTop': '10px'}),

        # TASK 2.3: Add two empty divisions for output inside the next inner division
        html.Div([
            html.Div([], id='fire-area-pie-chart'),
            html.Div([], id='vegetation-fire-bar-chart')
        ], style={'display': 'flex', 'justifyContent': 'space-around', 'marginTop': '20px'}),

    ]),
])

# TASK 2.4: Define @app.callback decorator
@app.callback(
    [Output(component_id='fire-area-pie-chart', component_property='children'),
     Output(component_id='vegetation-fire-bar-chart', component_property='children')],
    [Input(component_id='region-selector', component_property='value'),
     Input(component_id='year-selector', component_property='value')]
)
# TASK 2.5: Define the callback function
def reg_year_display(input_region, input_year):
    # Filter data based on selected region and year
    region_data = df[df['Region'] == input_region]
    year_data = region_data[region_data['Year'] == input_year]

    # Plot one - Monthly Average Estimated Fire Area (Pie Chart)
    est_data = year_data.groupby('Month')['Estimated_fire_area'].mean().reset_index()
    fig1 = px.pie(
        est_data,
        names='Month',
        values='Estimated_fire_area',
        title=f"{input_region}: Monthly Average Estimated Fire Area in {input_year}"
    )

    # Plot two - Monthly Average Count of Pixels for Presumed Vegetation Fires (Bar Chart)
    veg_data = year_data.groupby('Month')['Count'].mean().reset_index()
    fig2 = px.bar(
        veg_data,
        x='Month',
        y='Count',
        title=f"{input_region}: Average Count of Pixels for Presumed Vegetation Fires in {input_year}",
        labels={'Count': 'Average Pixel Count'},
        color='Month'
    )

    return [dcc.Graph(figure=fig1), dcc.Graph(figure=fig2)]

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
